In [ ]:
import pandas as pd

df = pd.read_csv(
    "/content/data.csv",
    encoding="latin-1"
)

df.head()


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [ ]:
label_cols = [
    'HS', 'Abusive',
    'HS_Individual', 'HS_Group',
    'HS_Religion', 'HS_Race',
    'HS_Gender', 'HS_Physical',
    'HS_Weak', 'HS_Moderate', 'HS_Strong'
]

X = df['Tweet']
y = df[label_cols]


In [ ]:
alay = pd.read_csv("/content/new_kamusalay.csv", encoding="latin-1")
alay.columns


Index(['anakjakartaasikasik', 'anak jakarta asyik asyik'], dtype='object')

In [ ]:
alay = pd.read_csv("/content/new_kamusalay.csv", encoding="latin-1")

# rapikan nama kolom (hilangkan spasi)
alay.columns = alay.columns.astype(str).str.strip()

# kalau ternyata file tidak punya header yang benar, pakai ini:
if alay.shape[1] >= 2:
    col1, col2 = alay.columns[0], alay.columns[1]
    alay_dict = dict(zip(alay[col1].astype(str).str.strip(),
                         alay[col2].astype(str).str.strip()))
else:
    raise ValueError("new_kamusalay.csv harus punya minimal 2 kolom (slang, formal).")


In [ ]:
def normalize(text):
    if not isinstance(text, str):
        return text
    words = text.lower().split()
    return " ".join([alay_dict.get(w, w) for w in words])

df["Tweet"] = df["Tweet"].apply(normalize)


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_cols = [
    'HS','Abusive','HS_Individual','HS_Group',
    'HS_Religion','HS_Race','HS_Physical','HS_Gender',
    'HS_Weak','HS_Moderate','HS_Strong'
]

missing = [c for c in label_cols if c not in df.columns]
print("Missing:", missing)


Missing: []


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
texts = df["Tweet"].astype(str).tolist()
labels = df[label_cols].astype(int).values


In [ ]:
encodings = tokenizer(
    texts,
    truncation=True,
    padding=True,
    max_length=128
)


In [ ]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).float()  # float untuk multilabel
        return item


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_tmp, y_train, y_tmp = train_test_split(texts, labels, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=42)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

train_enc = tokenizer(X_train, truncation=True, padding=True, max_length=128)
val_enc   = tokenizer(X_val,   truncation=True, padding=True, max_length=128)
test_enc  = tokenizer(X_test,  truncation=True, padding=True, max_length=128)


In [ ]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).float()  # penting: float
        return item

train_ds = TweetDataset(train_enc, y_train)
val_ds   = TweetDataset(val_enc, y_val)
test_ds  = TweetDataset(test_enc, y_test)


In [ ]:
from transformers import AutoModelForSequenceClassification

label_cols = [...]  # pastikan ini sudah sama seperti sebelumnya

model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))          # sigmoid
    preds = (probs >= 0.5).astype(int)         # threshold 0.5 (bisa di-tuning)

    return {
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
    }


In [ ]:
!pip install -U transformers accelerate


In [ ]:
import transformers
print(transformers.__version__)


4.57.3


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    logging_steps=50,
    report_to="none"
)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))      # sigmoid
    preds = (probs >= 0.5).astype(int)     # threshold 0.5

    return {
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
        "precision_micro": precision_score(labels, preds, average="micro", zero_division=0),
        "recall_micro": recall_score(labels, preds, average="micro", zero_division=0),
    }


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-3960739219.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("len(label_cols) =", len(label_cols))
print("model.num_labels =", model.config.num_labels)


len(label_cols) = 1
model.num_labels = 1


In [ ]:
# cek nama kolom tweet Anda
print(df.columns[:20])

text_col = "Tweet"  # kalau ternyata nama kolomnya beda, ganti di sini

# ambil semua kolom selain teks sebagai kandidat label
candidate_cols = [c for c in df.columns if c != text_col]

# pilih yang nilainya hanya 0/1 (atau boolean) => label multi-label
label_cols = []
for c in candidate_cols:
    s = df[c].dropna()
    uniq = set(s.unique().tolist())
    if uniq.issubset({0, 1, 0.0, 1.0, True, False}):
        label_cols.append(c)

print("Jumlah label:", len(label_cols))
print("label_cols:", label_cols)


Index(['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak',
       'HS_Moderate', 'HS_Strong'],
      dtype='object')
Jumlah label: 12
label_cols: ['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong']


In [ ]:
texts = df[text_col].astype(str).tolist()
labels = df[label_cols].astype(int).values

print("labels shape:", labels.shape)  # harus (N, jumlah_label)


labels shape: (13169, 12)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_tmp, y_train, y_tmp = train_test_split(texts, labels, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=42)


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

train_enc = tokenizer(X_train, truncation=True, padding=True, max_length=128)
val_enc   = tokenizer(X_val,   truncation=True, padding=True, max_length=128)
test_enc  = tokenizer(X_test,  truncation=True, padding=True, max_length=128)


In [ ]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).float()
        return item

train_ds = TweetDataset(train_enc, y_train)
val_ds   = TweetDataset(val_enc, y_val)
test_ds  = TweetDataset(test_enc, y_test)

print("1 sample label shape:", train_ds[0]["labels"].shape)  # harus (jumlah_label,)


1 sample label shape: torch.Size([12])


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)

print("model.num_labels =", model.config.num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.num_labels = 12


In [ ]:
from sklearn.metrics import classification_report

pred = trainer.predict(test_ds)
logits = pred.predictions
y_true = pred.label_ids

probs = 1 / (1 + np.exp(-logits))
y_pred = (probs >= 0.5).astype(int)

print(classification_report(y_true, y_pred, target_names=label_cols, zero_division=0))


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: Target size (torch.Size([16, 12])) must be the same as input size (torch.Size([16, 1]))

# FINAL TES

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
Tesla T4


In [ ]:
# =========================
# 0) IMPORT
# =========================
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# =========================
# 1) LOAD DATA (sesuaikan path)
# =========================
df = pd.read_csv("/content/data.csv", encoding="latin-1")

# Pastikan kolom teks benar
print("Kolom-kolom awal:", df.columns[:20])

text_col = "Tweet"  # ganti jika nama kolom teks Anda berbeda (misal: 'tweet' / 'text')

# =========================
# 2) AUTO-DETECT LABEL COLUMNS (0/1)
# =========================
candidate_cols = [c for c in df.columns if c != text_col]

label_cols = []
for c in candidate_cols:
    s = df[c].dropna()
    # Skip kolom non-numerik besar (misal id/username) jika ada
    try:
        uniq = set(pd.to_numeric(s, errors="coerce").dropna().unique().tolist())
    except Exception:
        continue

    if len(uniq) > 0 and uniq.issubset({0, 1, 0.0, 1.0}):
        label_cols.append(c)

print("Jumlah label terdeteksi:", len(label_cols))
print("Label columns:", label_cols)

# Jika label_cols masih 1 atau 0, berarti format label di CSV Anda beda.
# Anda bisa kirim df.head() + df.columns ke saya untuk dibetulkan.

# =========================
# 3) BUILD texts & labels
# =========================
texts = df[text_col].astype(str).tolist()
labels = df[label_cols].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int).values

print("labels shape:", labels.shape)  # harus (N, jumlah_label)

# =========================
# 4) SPLIT 70/15/15
# =========================
X_train, X_tmp, y_train, y_tmp = train_test_split(
    texts, labels, test_size=0.30, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.50, random_state=42
)

print("Train/Val/Test:", len(X_train), len(X_val), len(X_test))

# =========================
# 5) TOKENIZER
# =========================
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

train_enc = tokenizer(X_train, truncation=True, padding=True, max_length=128)
val_enc   = tokenizer(X_val,   truncation=True, padding=True, max_length=128)
test_enc  = tokenizer(X_test,  truncation=True, padding=True, max_length=128)

# =========================
# 6) DATASET PyTorch
# =========================
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).float()  # multilabel => float
        return item

train_ds = TweetDataset(train_enc, y_train)
val_ds   = TweetDataset(val_enc, y_val)
test_ds  = TweetDataset(test_enc, y_test)

print("1 sample label shape:", train_ds[0]["labels"].shape)  # harus (jumlah_label,)

# =========================
# 7) MODEL (Multi-label)
# =========================
model = AutoModelForSequenceClassification.from_pretrained(
    "indolem/indobertweet-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)

print("model.num_labels:", model.config.num_labels)

# =========================
# 8) METRICS multi-label
# =========================
def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    probs = 1 / (1 + np.exp(-logits))      # sigmoid
    y_pred = (probs >= 0.5).astype(int)    # threshold default 0.5

    return {
        "f1_micro": f1_score(y_true, y_pred, average="micro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "precision_micro": precision_score(y_true, y_pred, average="micro", zero_division=0),
        "recall_micro": recall_score(y_true, y_pred, average="micro", zero_division=0),
    }

# =========================
# 9) TRAINING ARGUMENTS (versi Anda pakai eval_strategy)
# =========================
args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    logging_steps=50,
    report_to="none"
)

# =========================
# 10) TRAINER
# =========================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# =========================
# 11) TRAIN
# =========================
trainer.train()

# =========================
# 12) EVALUATE TEST
# =========================
test_result = trainer.evaluate(test_ds)
print("TEST RESULT:", test_result)

# =========================
# 13) CLASSIFICATION REPORT per label (bagus untuk BAB IV)
# =========================
pred = trainer.predict(test_ds)
logits = pred.predictions
y_true = pred.label_ids

probs = 1 / (1 + np.exp(-logits))
y_pred = (probs >= 0.5).astype(int)

print(classification_report(y_true, y_pred, target_names=label_cols, zero_division=0))


Kolom-kolom awal: Index(['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak',
       'HS_Moderate', 'HS_Strong'],
      dtype='object')
Jumlah label terdeteksi: 12
Label columns: ['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong']
labels shape: (13169, 12)
Train/Val/Test: 9218 1975 1976


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

model.num_labels: 12


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Precision Micro,Recall Micro
1,0.212200,0.204187,0.771228,0.575828,0.757427,0.785541
2,0.158000,0.181725,0.793912,0.620876,0.831783,0.759340
3,0.143600,0.179134,0.805908,0.640875,0.817887,0.794275


TEST RESULT: {'eval_loss': 0.17410537600517273, 'eval_f1_micro': 0.8068152703048936, 'eval_f1_macro': 0.6568416681356811, 'eval_precision_micro': 0.8341721854304636, 'eval_recall_micro': 0.7811957330687175, 'eval_runtime': 15.2708, 'eval_samples_per_second': 129.397, 'eval_steps_per_second': 8.12, 'epoch': 3.0}
               precision    recall  f1-score   support

           HS       0.89      0.88      0.89       815
      Abusive       0.91      0.90      0.91       748
HS_Individual       0.79      0.74      0.76       504
     HS_Group       0.76      0.70      0.73       311
  HS_Religion       0.81      0.65      0.72       121
      HS_Race       0.88      0.70      0.78       101
  HS_Physical       0.00      0.00      0.00        36
    HS_Gender       0.00      0.00      0.00        44
     HS_Other       0.81      0.84      0.82       536
      HS_Weak       0.79      0.71      0.75       478
  HS_Moderate       0.71      0.62      0.66       264
    HS_Strong       0.85  

**data uji 1**

In [ ]:
label_map = {
    "HS": "Ujaran Kebencian",
    "Abusive": "Bahasa Kasar / Abusif",
    "HS_Individual": "Ujaran Kebencian terhadap Individu",
    "HS_Group": "Ujaran Kebencian terhadap Kelompok",
    "HS_Religion": "Ujaran Kebencian berbasis Agama",
    "HS_Race": "Ujaran Kebencian berbasis Ras atau Etnis",
    "HS_Physical": "Ujaran Kebencian berbasis Kondisi Fisik atau Disabilitas",
    "HS_Gender": "Ujaran Kebencian berbasis Gender atau Orientasi Seksual",
    "HS_Other": "Ujaran Kebencian Lainnya (slander, penghinaan umum)",
    "HS_Weak": "Ujaran Kebencian Tingkat Rendah",
    "HS_Moderate": "Ujaran Kebencian Tingkat Sedang",
    "HS_Strong": "Ujaran Kebencian Tingkat Tinggi"
}


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# prediksi ulang (jika belum)
pred = trainer.predict(test_ds)
logits = pred.predictions
y_true = pred.label_ids

probs = 1 / (1 + np.exp(-logits))
y_pred = (probs >= 0.5).astype(int)

# ubah nama label ke versi lengkap
label_names_full = [label_map[l] for l in label_cols]

print(classification_report(
    y_true,
    y_pred,
    target_names=label_names_full,
    zero_division=0
))


                                                          precision    recall  f1-score   support

                                        Ujaran Kebencian       0.89      0.88      0.89       815
                                   Bahasa Kasar / Abusif       0.91      0.90      0.91       748
                      Ujaran Kebencian terhadap Individu       0.79      0.74      0.76       504
                      Ujaran Kebencian terhadap Kelompok       0.76      0.70      0.73       311
                         Ujaran Kebencian berbasis Agama       0.81      0.65      0.72       121
                Ujaran Kebencian berbasis Ras atau Etnis       0.88      0.70      0.78       101
Ujaran Kebencian berbasis Kondisi Fisik atau Disabilitas       0.00      0.00      0.00        36
 Ujaran Kebencian berbasis Gender atau Orientasi Seksual       0.00      0.00      0.00        44
     Ujaran Kebencian Lainnya (slander, penghinaan umum)       0.81      0.84      0.82       536
                   

In [ ]:
import torch

def predict_tweet(text, model, tokenizer, label_cols, label_map, threshold=0.5):
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    # pindahkan ke device model (CPU / GPU)
    device = model.device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    results = {}
    for i, label in enumerate(label_cols):
        if probs[i] >= threshold:
            label_name = label_map.get(label, label)  # fallback jika tidak ada di map
            results[label_name] = float(probs[i])

    return results


In [ ]:
tweet = "Dasar bodoh, otak udang, gak pantas hidup!"

hasil = predict_tweet(
    tweet,
    model,
    tokenizer,
    label_cols,
    label_map,
    threshold=0.5
)

print("Hasil prediksi:")
for k, v in hasil.items():
    print(f"- {k}: {v:.3f}")


Hasil prediksi:
- Ujaran Kebencian: 0.981
- Bahasa Kasar / Abusif: 0.980
- Ujaran Kebencian terhadap Individu: 0.956
- Ujaran Kebencian Lainnya (slander, penghinaan umum): 0.712
- Ujaran Kebencian Tingkat Rendah: 0.948
